In [ ]:
import os
import cv2
import glob
import torch
import shutil
import itertools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
from sklearn.model_selection import train_test_split

In [ ]:
train_path = '../input/siim-isic-melanoma-classification/jpeg/train'
test_path = '../input/siim-isic-melanoma-classification/jpeg/test'
new_train = '/kaggle/working/train/train'
new_test = '/kaggle/working/test/test'

os.mkdir('/kaggle/working/train')
os.mkdir('/kaggle/working/test')

os.mkdir('/kaggle/working/train/benign')
os.mkdir('/kaggle/working/train/malignant')
os.mkdir('/kaggle/working/test/benign')
os.mkdir('/kaggle/working/test/malignant')

In [ ]:
train_csv = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')[1:].reset_index(drop=True)
test_csv = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

benign = list(train_csv['image_name'][[j for i, j in zip(list(train_csv['target']), 
                                                         range(len(train_csv))) if i == 1][:400]])
malignant = list(train_csv['image_name'][[j for i, j in zip(list(train_csv['target']), 
                                                         range(len(train_csv))) if i == 0][:400]])

In [ ]:
X = np.array([malignant, benign]).reshape(1, -1)[0]

train_malignant = X[:300]
test_malignant = X[300:400]
train_benign = X[400:700]
test_benign = X[700:]

In [ ]:
main_path = '../input/siim-isic-melanoma-classification/jpeg/train'

for i in [['/train/malignant', train_malignant], ['/test/malignant', test_malignant],
          ['/train/benign', train_benign], ['/test/benign', test_benign]]:
    path = i[0]
    new_dataset = i[1]
    dataset = glob.iglob(os.path.join(main_path, '*.jpg'))
    
    for img in dataset:
        if img[54:-4] in new_dataset:
            shutil.copy(img, '/kaggle/working'+path)

In [ ]:
normalizer = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ColorJitter(),
        transforms.Pad(4),
        transforms.RandomAffine(49),
        transforms.ToTensor(),
        normalizer
    ]),
    
    'test': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        normalizer
    ])
}

In [ ]:
data_images = {
    'train': datasets.ImageFolder('/kaggle/working/train', data_transforms['train']),
    'test': datasets.ImageFolder('/kaggle/working/test', data_transforms['test']),
}

In [ ]:
data_loaders = {
    'train': DataLoader(data_images['train'], batch_size=32, shuffle=True, num_workers=0),
    'test': DataLoader(data_images['test'], batch_size=32, shuffle=True, num_workers=0),
}

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2),
).to(device)

In [ ]:
optimizer = Adam(model.fc.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_model(epochs, optimizer, criterion, model):
    for epoch in range(epochs):
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss, running_corrects = 0.0, 0

            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_images[phase])
            epoch_acc = running_corrects.double() / len(data_images[phase])

        print('Epoch', str(epoch+1) + '/' + str(epochs), 'loss:', epoch_loss, 'accuracy:', epoch_acc)
    return model

In [ ]:
model = train_model(3, optimizer, criterion, model)